In [3]:
import pandas as pd
import numpy as np
from typing import Optional, List, Callable, Any, Union, Dict
from itertools import product
from statistics import mean
from pathlib import Path
import gzip
import os
import matplotlib.pyplot as plt
#import seaborn as sns
import re
from scipy.stats import f_oneway

# Read dataset

In [4]:
def read_ds_gzip(path: Optional[Path]=None, ds: str = "TRAIN") -> pd.DataFrame:
    """Args:
        path (Optional[Path], optional): the path to read the dataset file. Defaults to /kaggle/input/the-insa-starcraft-2-player-prediction-challenge/{ds}.CSV.gz.
        ds (str, optional): the part to read (TRAIN or TEST), to use when path is None. Defaults to "TRAIN".

    Returns:
        pd.DataFrame:
    """
    with gzip.open(f'/kaggle/input/the-insa-starcraft-2-player-prediction-challenge/{ds}.CSV.gz' if path is None else path) as f:
        max_actions = max(( len( str(c).split(",")) for c in f.readlines() ))
        f.seek(0)
        _names = ["battleneturl", "played_race"] if "TRAIN" in ds else ["played_race"]
        _names.extend(range(max_actions - len(_names)))
        return pd.read_csv(f, names=_names, dtype= str)

def read_ds(path: Optional[Path]=None, ds: str = "TRAIN"):
    """Args:
        path (Optional[Path], optional): the path to read the dataset file. Defaults to /kaggle/input/the-insa-starcraft-2-player-prediction-challenge/{ds}.CSV.gz.
        ds (str, optional): the part to read (TRAIN or TEST), to use when path is None. Defaults to "TRAIN".

    Returns:
        pd.DataFrame:
    """
    with open(f'/kaggle/input/train-sc2-keystrokes/{ds}.CSV' if path is None else path) as f:
        max_actions = max(( len( str(c).split(",")) for c in f.readlines() ))
        f.seek(0)
        _names = ["battleneturl", "played_race"] if "TRAIN" in ds else ["played_race"]
        _names.extend(range(max_actions - len(_names)))
        return pd.read_csv(f, names=_names, dtype= str)

In [5]:
features_train = read_ds(Path(os.path.abspath('')) / "data/train.csv") # Replace with correct path 
# features_test = read_ds("TEST")
features_train.shape #, features_test.shape

(3052, 10539)

In [6]:
features_train

,battleneturl,played_race,0,1,2,3,4,5,6,7,...,10527,10528,10529,10530,10531,10532,10533,10534,10535,10536
0,http://eu.battle.net/sc2/en/profile/4234852/1/...,Protoss,Base,s,s,s,s,s,t5,Base,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://eu.battle.net/sc2/en/profile/3074362/1/...,Protoss,s,Base,s,s,Base,s,s,Base,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://eu.battle.net/sc2/en/profile/4234852/1/...,Protoss,Base,s,s,s,Base,s,hotkey30,hotkey00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://eu.battle.net/sc2/en/profile/3074362/1/...,Protoss,Base,s,s,Base,s,s,s,t5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://eu.battle.net/sc2/en/profile/4234852/1/...,Protoss,Base,s,s,s,Base,s,hotkey30,hotkey00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3047,http://xx.battle.net/sc2/en/profile/405/1/MMA/,Terran,s,s,s,s,s,hotkey10,hotkey20,hotkey30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3048,http://xx.battle.net/sc2/en/profile/410/1/STBo...,Terran,s,s,hotkey10,s,hotkey20,s,s,hotkey12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3049,http://xx.battle.net/sc2/en/profile/405/1/MMA/,Terran,s,s,s,hotkey10,hotkey20,hotkey30,hotkey40,hotkey50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3050,http://xx.battle.net/sc2/en/profile/410/1/STBo...,Terran,s,s,hotkey10,s,hotkey20,s,s,hotkey12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
features_train.isna().sum()

battleneturl       0
played_race        0
0                  8
1                  8
2                  8
                ... 
10532           3051
10533           3051
10534           3051
10535           3051
10536           3051
Length: 10539, dtype: int64

# Pre-processing and feature creation

Step to take out outliers: defined as the rows which had null values in the first action column (0), which also had null values in the first 5 action columns. Resultig dataset: 10539 rows (games)

In [8]:
df_try= features_train.copy()

In [9]:
df_try.head(3)

,battleneturl,played_race,0,1,2,3,4,5,6,7,...,10527,10528,10529,10530,10531,10532,10533,10534,10535,10536
0,http://eu.battle.net/sc2/en/profile/4234852/1/...,Protoss,Base,s,s,s,s,s,t5,Base,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://eu.battle.net/sc2/en/profile/3074362/1/...,Protoss,s,Base,s,s,Base,s,s,Base,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://eu.battle.net/sc2/en/profile/4234852/1/...,Protoss,Base,s,s,s,Base,s,hotkey30,hotkey00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#df_cleaned = features_train.dropna(subset=[0, 1, 2, 4],inplace= True)

#df_cleaned.shape


Function to create the dummie features for played race

In [11]:
def get_dummies(df: pd.DataFrame):
    """Converts textual columns to one-hot encoded vectors (one column per possible value)"""
    df = pd.get_dummies(df, columns=["played_race"])
    return df

Convertion of players urls to codes:

In [12]:
def to_categories(df: pd.DataFrame, col: str="battleneturl") -> None:
    """Convert col of df to a categorical column"""
    df["battleneturl"] = pd.Categorical(df["battleneturl"])
    df[[col]] = df[[col]].apply(lambda x: x.cat.codes)

### Calculate the features: s_ratio, base_ratio and mineral_ratio:

In [13]:
def calculate_action_ratios(row):
    # Define the columns of actions (columns 0 to 10536, excluding the last column 'num_actions')
    #action_columns = [str(i) for i in range(0, 10537)]
    
    # Extract action values from the row
    actions = row.values
    
    # Count occurrences of 's', 'Base', and 'SingleMineral'
    s_count = sum(1 for action in actions if action == 's')
    base_count = sum(1 for action in actions if action == 'Base')
    mineral_count = sum(1 for action in actions if action == 'SingleMineral')
    
    # Extract the total number of actions from the 'num_actions' column
    total_actions = sum(1 for action in actions)
    
    # Calculate the ratios
    ratio_s = s_count / total_actions if total_actions > 0 else 0
    ratio_base = base_count / total_actions if total_actions > 0 else 0
    ratio_mineral = mineral_count / total_actions if total_actions > 0 else 0
    
    # Return the ratios as a dictionary
    return pd.Series({'ratio_s': ratio_s, 'ratio_base': ratio_base, 'ratio_mineral': ratio_mineral})

### Calculate the feature number of actions per interval

In [14]:
def calculate_action_mean(row):
    """
    Calculate the mean number of actions per 5 seconds for a given row.

    Args:
        row (pd.Series): The row to process.

    Returns:
        float: The mean number of actions per 5 seconds.
    """
    counts = []
    count = 0

    for value in row:
        if pd.isna(value):  # Stop if the value is NaN
            break
        elif isinstance(value, str) and re.match(r"^t\d+$", value):  # Found a time window
            if count > 0:
                counts.append(count)  # Save the current count
            count = 0  # Restart the count
        else:  # Count non-NaN, non-time values
            count += 1

    if count > 0:  # Add any remaining count
        counts.append(count)

    return np.mean(counts) if counts else 0  # Calculate the mean, default to 0 if no counts

In [15]:
def count_values(row):
    return row.value_counts()

### Calculate ratio of kinds of hotkeys

In [16]:

def count_hotkeys_xx(row):
    # Find all hotkeyXX patterns
    hotkeys = re.findall(r"hotkey\d{2}", " ".join(row.astype(str)))
    
    # Count occurrences of all hotkeys
    hotkey_counts = pd.Series(hotkeys).value_counts()
    
    # Calculate the total number of hotkeys
    total_hotkeys = hotkey_counts.sum()
    
    # Filter for hotkeyX0 patterns
    hotkey_x0_counts = hotkey_counts[hotkey_counts.index.str.endswith("0")].sum()
    # Filter for hotkeyX1 patterns
    hotkey_x1_counts = hotkey_counts[hotkey_counts.index.str.endswith("1")].sum()
    # Filter for hotkeyX2 patterns
    hotkey_x2_counts = hotkey_counts[hotkey_counts.index.str.endswith("2")].sum()
    
    # Calculate the ratio of hotkeyX0 to all hotkeys
    ratio_x0 = hotkey_x0_counts / total_hotkeys if total_hotkeys > 0 else 0
    # Calculate the ratio of hotkeyX0 to all hotkeys
    ratio_x1 = hotkey_x1_counts / total_hotkeys if total_hotkeys > 0 else 0
    # Calculate the ratio of hotkeyX0 to all hotkeys
    ratio_x2 = hotkey_x2_counts / total_hotkeys if total_hotkeys > 0 else 0
   # Return the ratios as a dictionary
    return pd.Series({'ratio_x0': ratio_x0, 'ratio_x1': ratio_x1, 'ratio_x2': ratio_x2})



Justify the presence of 0s in the ratio of hotkeyX1 by looking at the graph.

### Number of all keys pressed per second

In [17]:
def count_values(row):
    # Extract all values of the format "t<number>"
    t_values = [int(value[1:]) for value in row if re.match(r"^t\d+$", str(value))]
    
    # Determine the last "t<number>" value, default to 165 if not found
    divisor = t_values[-1] if t_values else 5
    
    # Filter out "t<number>" values
    filtered_values = [value for value in row if not re.match(r"^t\d+$", str(value))]
    
    # Count occurrences of each value in the filtered list
    value_counts = pd.Series(filtered_values).value_counts()
    
    # Divide by the determined divisor
    return value_counts / divisor



### Count of all keys pressed in the first 5 seconds:


In [18]:
# Function to count values up to the first occurrence of "t5"
def count_values_t5(row):
    # Slice the row up to (but not including) the first "t5"
    sliced_row = row[:row.tolist().index("t5")] if "t5" in row.tolist() else row
    # Count occurrences of each value in the sliced row
    value_counts = sliced_row.value_counts().to_dict()
    return value_counts

def count5(data):
    value_counts_df = pd.json_normalize(data.apply(count_values_t5, axis=1))
    value_counts_df = value_counts_df.fillna(0)
    value_counts_df = value_counts_df.add_suffix("_f5")
    return value_counts_df

In [19]:
df_teste= count5(df_try)

In [20]:
df_teste.columns

Index(['s_f5', 'http://eu.battle.net/sc2/en/profile/4234852/1/First/_f5',
       'Protoss_f5', 'Base_f5',
       'http://eu.battle.net/sc2/en/profile/3074362/1/Stardust/_f5',
       'hotkey30_f5', 'hotkey00_f5', 'hotkey32_f5',
       'http://eu.battle.net/sc2/en/profile/3401218/1/Welmu/_f5',
       'hotkey40_f5',
       ...
       'http://xx.battle.net/sc2/en/profile/404/1/sOs/_f5',
       'http://xx.battle.net/sc2/en/profile/402/1/Dear/_f5',
       'http://xx.battle.net/sc2/en/profile/408/1/Polt/_f5',
       'http://xx.battle.net/sc2/en/profile/414/1/ducKDeoK/_f5',
       'http://xx.battle.net/sc2/en/profile/413/1/Maru/_f5',
       'http://xx.battle.net/sc2/en/profile/406/1/INnoVation/_f5',
       'http://xx.battle.net/sc2/en/profile/410/1/STBomber/_f5',
       'http://xx.battle.net/sc2/en/profile/407/1/NaNiwa/_f5',
       'http://xx.battle.net/sc2/en/profile/401/1/Soulkey/_f5',
       'http://xx.battle.net/sc2/en/profile/405/1/MMA/_f5'],
      dtype='object', length=233)

### Creation of features

In [30]:
def create_features(dataset):
    """
    Create features for the given dataset by applying feature engineering functions.

    Args:
        dataset (pd.DataFrame): The dataset to process.

    Returns:
        pd.DataFrame: The dataset with new features added.
    """
    # Ensure a copy of the dataset is used to avoid modifying the original
    df_features = dataset.copy()
    df_temp= dataset.drop(['battleneturl','played_race'],axis=1, errors='ignore')
    print(df_temp)

    # Apply the calculate_action_ratios function
    action_ratios = df_temp.apply(calculate_action_ratios, axis=1)
    df_features[['ratio_s', 'ratio_base', 'ratio_mineral']] = action_ratios

    # Apply the calculate_action_mean function
    df_features['action_per_5_seconds'] = df_temp.iloc[:, 2:].apply(calculate_action_mean, axis=1)

    # Apply the count_hotkeys_xx function
    hotkeys_ratios = df_temp.apply(count_hotkeys_xx, axis=1)
    df_features[['ratio_x0', 'ratio_x1', 'ratio_x2']] = hotkeys_ratios

    #Number of all keys pressed per second
    df_features[['Base', 'SingleMineral', 'hotkey00', 'hotkey01', 'hotkey02', 'hotkey10',
       'hotkey11', 'hotkey12', 'hotkey20', 'hotkey21', 'hotkey22', 'hotkey30',
       'hotkey31', 'hotkey32', 'hotkey40', 'hotkey41', 'hotkey42', 'hotkey50',
       'hotkey51', 'hotkey52', 'hotkey60', 'hotkey61', 'hotkey62', 'hotkey70',
       'hotkey71', 'hotkey72', 'hotkey80', 'hotkey81', 'hotkey82', 'hotkey90',
       'hotkey91', 'hotkey92', 's']] = df_temp.apply(count_values, axis=1).fillna(0)
    
    # Count of all keys pressed in the first 5 seconds:
    df_features[['s_f5', 'Base_f5', 'hotkey30_f5', 'hotkey00_f5', 'hotkey32_f5',
       'hotkey40_f5', 'hotkey10_f5', 'hotkey60_f5', 'hotkey90_f5',
       'hotkey20_f5', 'hotkey62_f5', 'hotkey80_f5', 'hotkey70_f5',
       'hotkey50_f5', 'SingleMineral_f5', 'hotkey12_f5', 'hotkey11_f5',
       'hotkey21_f5', 'hotkey31_f5', 'hotkey41_f5', 'hotkey51_f5',
       'hotkey61_f5', 'hotkey71_f5', 'hotkey52_f5', 'hotkey42_f5',
       'hotkey22_f5', 'hotkey92_f5', 'hotkey02_f5', 'hotkey82_f5',
       'hotkey72_f5']]= count5(df_temp)

    return df_features

In [22]:
def preprocess(df: pd.DataFrame,  is_train: bool=True, convert_race: bool=True):
    """Calls FeatureGetter on the dataframe, applying preprocessing steps before
    Args:
        df (pd.DataFrame)
        min_ (int)
        max_ (int)
        is_train (bool, optional): whether the current dataframe contains training data 
            (to preprocess dependent variable or not). Defaults to True.
        convert_race (bool, optional): whether to convert race attribute to dummies. Defaults to True.

    Returns:
        _type_: _description_
    """
    df_feat = create_features(df)
    df_cat= df[['battleneturl','played_race']]
    if convert_race: 
        df = get_dummies(df)
    df.columns = df.columns.astype(str)
    if is_train:
        to_categories(df)

    #merged_df = df_feat.merge(df, left_index=True, right_index=True)
    return df_feat,df 

### Preprocess test set

In [27]:
def read_ds(path: Optional[Path]=None, ds: str = "TRAIN"):
    """Args:
        path (Optional[Path], optional): the path to read the dataset file. Defaults to /kaggle/input/the-insa-starcraft-2-player-prediction-challenge/{ds}.CSV.gz.
        ds (str, optional): the part to read (TRAIN or TEST), to use when path is None. Defaults to "TRAIN".

    Returns:
        pd.DataFrame:
    """
    with open(f'/kaggle/input/train-sc2-keystrokes/{ds}.CSV' if path is None else path) as f:
        max_actions = max(( len( str(c).split(",")) for c in f.readlines() ))
        f.seek(0)
        _names = ["played_race"] if "TRAIN" in ds else ["played_race"]
        _names.extend(range(max_actions - len(_names)))
        return pd.read_csv(f, names=_names, dtype= str)

In [28]:
#read test set 
features_test = read_ds("data/TEST.csv")
print(features_test.shape)
features_test.head(5)

(340, 6818)


,played_race,0,1,2,3,4,5,6,7,8,...,6807,6808,6809,6810,6811,6812,6813,6814,6815,6816
0,Zerg,s,s,s,hotkey10,s,hotkey60,s,hotkey00,s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Terran,s,hotkey30,hotkey00,t5,s,hotkey32,s,hotkey32,s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Protoss,Base,s,s,Base,s,hotkey20,hotkey30,hotkey60,hotkey40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Zerg,s,s,s,s,hotkey20,s,hotkey40,t5,t10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Protoss,Base,s,s,Base,s,hotkey50,hotkey30,t5,hotkey40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_test, df = preprocess(features_test)

     0         1         2         3         4         5         6     \
0       s         s         s  hotkey10         s  hotkey60         s   
1       s  hotkey30  hotkey00        t5         s  hotkey32         s   
2    Base         s         s      Base         s  hotkey20  hotkey30   
3       s         s         s         s  hotkey20         s  hotkey40   
4    Base         s         s      Base         s  hotkey50  hotkey30   
..    ...       ...       ...       ...       ...       ...       ...   
335     s         s         s         s  hotkey10  hotkey30         s   
336  Base         s         s         s      Base         s  hotkey10   
337     s         s  hotkey10         s  hotkey20         s         s   
338     s         s         s  hotkey10  hotkey20  hotkey30  hotkey40   
339     s         s  hotkey10         s  hotkey20         s         s   

         7         8         9     ... 6807 6808 6809 6810 6811 6812 6813  \
0    hotkey00         s  hotkey90  ...  NaN  N

ValueError: Columns must be same length as key

In [ ]:
#df_features,df_cat= preprocess(features_train)

In [22]:
df_features.columns

Index(['battleneturl',  'played_race',              0,              1,
                    2,              3,              4,              5,
                    6,              7,
       ...
        'hotkey51_f5',  'hotkey61_f5',  'hotkey71_f5',  'hotkey52_f5',
        'hotkey42_f5',  'hotkey22_f5',  'hotkey92_f5',  'hotkey02_f5',
        'hotkey82_f5',  'hotkey72_f5'],
      dtype='object', length=10609)

In [23]:
df_features.describe()

,ratio_s,ratio_base,ratio_mineral,action_per_5_seconds,ratio_x0,ratio_x1,ratio_x2,Base,SingleMineral,hotkey00,...,hotkey51_f5,hotkey61_f5,hotkey71_f5,hotkey52_f5,hotkey42_f5,hotkey22_f5,hotkey92_f5,hotkey02_f5,hotkey82_f5,hotkey72_f5
count,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000,...,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000,3052.000000
mean,0.069942,0.002203,0.000531,11.400500,0.043936,0.008399,0.945044,0.022977,0.005726,0.001104,...,0.006553,0.007536,0.002621,0.123853,0.280472,0.280799,0.022608,0.022936,0.000328,0.000655
std,0.042995,0.003320,0.001939,3.215464,0.041450,0.014217,0.064155,0.036394,0.023119,0.003964,...,0.080699,0.086497,0.051139,0.604578,1.027309,0.907443,0.304796,0.329075,0.018101,0.036202
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.040714,0.000190,0.000000,8.921677,0.026018,0.000000,0.937426,0.001778,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.059600,0.001139,0.000095,11.236204,0.037794,0.000000,0.954920,0.011915,0.000810,0.000524,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.088166,0.002752,0.000285,13.576655,0.053413,0.013534,0.966400,0.026855,0.002410,0.001527,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.312043,0.029420,0.025624,23.573248,1.000000,0.100000,0.993002,0.600000,0.400000,0.200000,...,1.000000,1.000000,1.000000,6.000000,8.000000,8.000000,6.000000,7.000000,1.000000,2.000000


In [24]:
df_cat

,battleneturl,0,1,2,3,4,5,6,7,8,...,10530,10531,10532,10533,10534,10535,10536,played_race_Protoss,played_race_Terran,played_race_Zerg
0,53,Base,s,s,s,s,s,t5,Base,s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
1,29,s,Base,s,s,Base,s,s,Base,s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
2,53,Base,s,s,s,Base,s,hotkey30,hotkey00,t5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
3,29,Base,s,s,Base,s,s,s,t5,Base,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
4,53,Base,s,s,s,Base,s,hotkey30,hotkey00,t5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3047,192,s,s,s,s,s,hotkey10,hotkey20,hotkey30,hotkey40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False
3048,196,s,s,hotkey10,s,hotkey20,s,s,hotkey12,hotkey22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False
3049,192,s,s,s,hotkey10,hotkey20,hotkey30,hotkey40,hotkey50,hotkey60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False
3050,196,s,s,hotkey10,s,hotkey20,s,s,hotkey12,hotkey22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False
